# INTELIGENCIA ARTIFICIAL (1INF24)
Dr. Edwin Villanueva Talavera (ervillanueva@pucp.edu.pe)

# Agente de Búsqueda sin Información en el Entorno del 8-puzzle
El presente notebook aborda la creación de un agente de búsqueda sin información para resolver el entorno de juego 8-puzzle.

<font color='orange'>Entorno del 8-puzzle</font>

<img src='https://images-na.ssl-images-amazon.com/images/I/61x5wYJJtsL._SX425_.jpg' width=200px>


## Definición del Entorno

### Clase <b>Thing</b>
Esta clase genérica representa cualquier objeto físico que puede aparecer en un <b>Ambiente</b>. (No editar)

In [2]:
class Thing(object):
    def is_alive(self):
        """Cosas 'vivas' deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")


### Clase <b>Environment</b>
Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno típicamente se necesitará implementar 2 cosas: <b>percept</b>, que define la percepción que el agente ve; y <b>execute_action</b>, que define los efectos de ejecutar una acción. El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [3]:
class Environment(object):
    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localizacion por defecto para colocar una nueva cosa sin localizacion especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Anhade una cosa thing al entorno en la localizacion location.
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)

        assert thing not in self.things, "No permite anhadir la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta)."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()


### Clase <b>EightpuzzleEnvironment</b>
En esta clase se implementa el entorno del 8-puzzle. Un agente en este entorno percibe el estado del entorno como una tupla de tuplas de 9 numeros, siendo el numero 0 la posicion vacia. Para la Figura de arriba el estado es: ((1, 2, 3), (4, 5, 6), (7, 8, 0))

In [4]:
import numpy as np
from copy import deepcopy

class EightpuzzleEnvironment(Environment):
    def __init__(self, initial_state):
        super().__init__()
        self.status = initial_state

    def thing_classes(self):
        return [EightpuzzleReflexAgent]

    def percept(self, agent):
        """Retorna el estado del ambiente (las piezas que estan en cada posicion)."""
        return self.status

    def execute_action(self, agent, action):
        """Implementa el Mapa De Transicion: Cambia la posicion de las piezas de acuerdo a la accion solicitada del blanco;
        Cada accion valida debe provocar una disminución de desempeño en 1 unidad."""
        state = np.array(self.status)  # Convierte el estado del entorno (tuple inmutable) en un array numpy para manipulacion

        tmp = np.where(state == 0)  # Obtiene las coordenadas del casillero en blanco (0)
        i_blank = tmp[0][0]  # Fila donde esta el blanco
        j_blank = tmp[1][0]  # Columna donde esta el blanco

        new_state = deepcopy(state)  # Copia state en nueva variable newState

        if action == 'Right':
            if j_blank < 2:
                new_state[i_blank, j_blank], new_state[i_blank, j_blank + 1] = state[i_blank, j_blank + 1], state[i_blank, j_blank]
                agent.performance -= 1
        elif action == 'Left':
            if j_blank > 0:
                new_state[i_blank, j_blank], new_state[i_blank, j_blank - 1] = state[i_blank, j_blank - 1], state[i_blank, j_blank]
                agent.performance -= 1
        elif action == 'Up':
            if i_blank > 0:
                new_state[i_blank - 1, j_blank], new_state[i_blank, j_blank] = state[i_blank, j_blank], state[i_blank - 1, j_blank]
                agent.performance -= 1
        elif action == 'Down':
            if i_blank < 2:
                new_state[i_blank + 1, j_blank], new_state[i_blank, j_blank] = state[i_blank, j_blank], state[i_blank + 1, j_blank]
                agent.performance -= 1

        self.status = tuple(map(tuple, new_state))  # Actualiza el estado (como tupla de tuplas)


## Definiciones de Problema de Búsqueda y Algoritmos de Búsqueda Ciega

### Clase <b>SearchProblem</b>
Esta es una clase abstracta para definir problemas de búsqueda. Se debe hacer subclases que implementen los métodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de búsqueda.

In [5]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede anhadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con
        la accion action, asumiendo un costo c para llegar hasta state1.
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1


### Clase <b>EightpuzzleSearchProblem</b>
Esta es una subclase de SearchProblem donde se definirá concretamente el problema de búsqueda para el ambiente del 8-puzzle. Se necesita completar Actions (acciones disponibles para un estado dado) y result (que estado resulta de ejecutar una acción en un estado).

In [6]:
class EightpuzzleSearchProblem(SearchProblem):
    def __init__(self, initial, goal):
        """El constructor recibe el estado inicial y el estado objetivo."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones ejecutables desde el estado state.
        Por ejemplo, para el estado ((0, 1, 2), (3, 4, 5), (6, 7, 8)) debe retornar: acciones = ['Down', 'Right']"""
        state = np.array(state)  # Convierte el state (tuple inmutable) a un array numpy para manipulacion

        acciones = []
        tmp = np.where(state == 0)  # Obtiene las coordenadas del casillero en blanco (0)
        i_blank = tmp[0][0]  # Fila donde esta el blanco
        j_blank = tmp[1][0]  # Columna donde esta el blanco

        if j_blank < 2:
            acciones.append('Right')
        if j_blank > 0:
            acciones.append('Left')
        if i_blank > 0:
            acciones.append('Up')
        if i_blank < 2:
            acciones.append('Down')

        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion action desde state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, para state = ((0, 1, 2), (3, 4, 5), (6, 7, 8)) y action = 'Right' debe retornar newState = ((0, 1, 2), (3, 4, 5), (6, 7, 8))"""
        state = np.array(state)  # Convierte el state (tuple inmutable) a un array numpy para manipulacion

        tmp = np.where(state == 0)  # Obtiene las coordenadas del casillero en blanco (0)
        i_blank = tmp[0][0]  # Fila donde esta el blanco
        j_blank = tmp[1][0]  # Columna donde esta el blanco

        new_state = deepcopy(state)  # Copia state en variable newState

        if action == 'Right':
            if j_blank < 2:
                new_state[i_blank, j_blank], new_state[i_blank, j_blank + 1] = state[i_blank, j_blank + 1], state[i_blank, j_blank]

        elif action == 'Left':
            if j_blank > 0:
                new_state[i_blank, j_blank], new_state[i_blank, j_blank - 1] = state[i_blank, j_blank - 1], state[i_blank, j_blank]

        elif action == 'Up':
            if i_blank > 0:
                new_state[i_blank - 1, j_blank], new_state[i_blank, j_blank] = state[i_blank, j_blank], state[i_blank - 1, j_blank]

        elif action == 'Down':
            if i_blank < 2:
                new_state[i_blank + 1, j_blank], new_state[i_blank, j_blank] = state[i_blank, j_blank], state[i_blank + 1, j_blank]

        return tuple(map(tuple, new_state))  # El estado retornado es una tupla

    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state)

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action
        El costo del camino para llegar a state1 es c. El costo de la accion es = 1"""
        return c + 1;


### Clase <b>Node</b>
Estructura de datos para almacenar la información de un nodo en un <b>árbol de búsqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. También incluye la acción que nos llevó al presente nodo y el costo total del camino desde el nodo raíz hasta este nodo.

In [7]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

        #milista = []
        #for action in problem.actions(self.state)
        #    milista.append(self.child_node(problem, action))

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __lt__(self, node):
        return self.state < node.state

    def __eq__(self, other):
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __hash__(self):
        return hash(self.state)


### <b> Define una cola tipo FIFO First-In-First-Out (para BFS)</b>

In [8]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()


### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>
Algoritmo de general de búsqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si frontier es tipo FIFO hace búsqueda en amplitud (BFS), si la frontier es una pila hará búsqueda en profundidad (DFS). Devuelve el nodo solución y una lista de nodos visitados durante la búsqueda.

In [9]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()    # Memoria de estados visitados
    visited_nodes = []  # Almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)

        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None


## Definición del Agente de Búsqueda en el entorno 8-puzzle

### Clase <b>Agent</b>
Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la función que implementa el <b>programa del agente</b>. Esta función debe tomar como argumento la <b>percepción</b> del agente y debe retornar una <b>acción</b>. La definición de Percepción y Acción depende del ambiente de trabajo (environment) donde el agente existe. El agente también puede tener el slot <b>.performance</b>, que guarda el desempeño del agente en su ambiente (desempeño visto desde el agente).

In [10]:
import collections
import random

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.abc.Callable)
        self.program = program


### <b>Clase que implementa el programa del agente que busca y ejecuta soluciones en el entorno 8-puzzle</b>

In [11]:
class EightpuzzleSearchProgram:
    def __init__(self, goal_state, search_method):
        self.goal = goal_state
        self.method = search_method
        self.seq = []  # Lista de acciones a ejecutar, inicialmente vacia

    def __call__(self, percept):
        state = percept
        if state == self.goal:  # Si el ambiente esta en el estado objetivo no hace nada
            return 'None'
        if not self.seq:  # Si la lista de acciones esta vacia
            print('Agente buscara solución al 8-puzzle: estado_inicial = {}. estado_objetivo={}'.format(state,self.goal))
            search_problem = EightpuzzleSearchProblem(state, self.goal)
            if self.method == 'bfs':
                goal_node, visited_nodes = graph_search(search_problem, FIFOQueue())  # Frontera es una cola FIFO
            elif self.method == 'dfs':
                goal_node, visited_nodes = graph_search(search_problem, [])  # Frontera es una pila ([] es una pila en Python)
            else:
                raise NotImplementedError

            if goal_node == None:  # Sin solucion
                print('No se encontró solución para el 8-puzzle con metodo {}'.format(self.method))
                return 'None'

            self.seq = goal_node.solution()
            print('Agente planeó una solución al 8-puzzle con {}: Seq = {}. Nodos visitados={}. Costo Solución = {}'.format(self.method, self.seq, len(visited_nodes),goal_node.path_cost))

        action = self.seq.pop(0)
        return action


### Probando el entorno 8-puzzle y agente de búsqueda

In [12]:
"""Crea el entorno del 8-puzzle con estado inicial 'initial' """
initial = ((3, 1, 2), (6, 4, 5), (7, 8, 0))
goal    = ((0, 1, 2), (3, 4, 5), (6, 7, 8))
e = EightpuzzleEnvironment(initial)

In [13]:
p = EightpuzzleSearchProgram(goal, 'bfs')

In [14]:
"""Crea un agente de busqueda para alcanzar goalstate"""
a = Agent(EightpuzzleSearchProgram(goal, 'bfs'))

"""Anhade el agente creado al entorno"""
e.add_thing(a)

In [15]:
# Ejecuta el entorno 25 pasos
e.run(25)

Agente buscara solución al 8-puzzle: estado_inicial = ((3, 1, 2), (6, 4, 5), (7, 8, 0)). estado_objetivo=((0, 1, 2), (3, 4, 5), (6, 7, 8))
Agente planeó una solución al 8-puzzle con bfs: Seq = ['Left', 'Left', 'Up', 'Up']. Nodos visitados=17. Costo Solución = 4


In [16]:
# Muestra el estado del entorno
e.status

((np.int64(0), np.int64(1), np.int64(2)),
 (np.int64(3), np.int64(4), np.int64(5)),
 (np.int64(6), np.int64(7), np.int64(8)))